In [ ]:
!pip install python-telegram-bot

In [ ]:
import os
import logging
import tensorflow as tf
from telegram import Update
from telegram.ext import Application, CommandHandler, MessageHandler, filters, ContextTypes
from PIL import Image
import numpy as np
import nest_asyncio

nest_asyncio.apply()

logging.basicConfig(format="%(asctime)s - %(name)s - %(levelname)s - %(message)s", level=logging.INFO)

MODEL_PATH = "/content/drive/MyDrive/PyLearn/Assignment_53_7.3/2 - 17 flowers/models/model with augmentation.h5"

model = tf.keras.models.load_model(MODEL_PATH)

IMG_SIZE = (224, 224)

class_labels = {
    'bluebell': 0, 'buttercup': 1, 'coltsfoot': 2, 'cowslip': 3, 'crocus': 4,
    'daffodil': 5, 'daisy': 6, 'dandelion': 7, 'fritillary': 8, 'iris': 9,
    'lilyvalley': 10, 'pansy': 11, 'snowdrop': 12, 'sunflower': 13, 'tigerlily': 14,
    'tulip': 15, 'windflower': 16
}

idx_to_class = {v: k for k, v in class_labels.items()}

def preprocess_image(image_path):
    img = Image.open(image_path).convert("RGB")
    img = img.resize((256, 256))
    img_array = np.array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

def predict(image_path):
    img_array = preprocess_image(image_path)
    predictions = model.predict(img_array)
    predicted_class_idx = np.argmax(predictions)
    predicted_class_name = idx_to_class[predicted_class_idx]
    return predicted_class_name


async def start(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    await update.message.reply_text("سلام! عکس گل خود را ارسال کنید.")

async def handle_photo(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    photo = await update.message.photo[-1].get_file()
    image_path = "received.jpg"
    await photo.download_to_drive(image_path)

    prediction = predict(image_path)
    await update.message.reply_text(f"نتیجه پیش‌بینی: {prediction}")



def main():
    TOKEN = "TOKEN"
    app = Application.builder().token(TOKEN).build()

    app.add_handler(CommandHandler("start", start))
    app.add_handler(MessageHandler(filters.PHOTO, handle_photo))

    app.run_polling()

if __name__ == "__main__":
    main()